# CIFAR-10

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt

In [2]:
(x_train, y_train), (x_test, y_test) = tfds.as_numpy(tfds.load("cifar10",
                                                               split=['train', 'test'],
                                                               as_supervised=True,
                                                               shuffle_files=True,
                                                               batch_size=-1))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteY4L7Z5/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteY4L7Z5/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
# Show examples of dataset
#fig = tfds.show_examples(ds, info)

In [3]:
x_train, x_test = x_train / 255., x_test / 255.
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000,), (10000, 32, 32, 3), (10000,))

In [4]:
# Get the number of classes
classes = len(set(y_train))
classes

10

In [ ]:
# Build a neural network (Improved)
i = tf.keras.layers.Input(shape=x_train[0].shape)
# First group
x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(i)  # Without same padding the layers would shrink
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
# Second group
x = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x) 
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
# Third group
x = tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x) 
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(classes, activation="softmax")(x)

model = tf.keras.Model(i, output)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
r = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

Epoch 1/50
1563/1563 [==============================] - 19s 6ms/step - loss: 1.3154 - accuracy: 0.5467 - val_loss: 1.0692 - val_accuracy: 0.6221
Epoch 2/50
1563/1563 [==============================] - 9s 6ms/step - loss: 0.8492 - accuracy: 0.7071 - val_loss: 0.7796 - val_accuracy: 0.7312
Epoch 3/50
1563/1563 [==============================] - 9s 6ms/step - loss: 0.6877 - accuracy: 0.7633 - val_loss: 0.7815 - val_accuracy: 0.7430
Epoch 4/50
1563/1563 [==============================] - 9s 6ms/step - loss: 0.5791 - accuracy: 0.8001 - val_loss: 0.6704 - val_accuracy: 0.7755
Epoch 5/50
1563/1563 [==============================] - 9s 6ms/step - loss: 0.4916 - accuracy: 0.8310 - val_loss: 0.6257 - val_accuracy: 0.7934
Epoch 6/50
1563/1563 [==============================] - 9s 6ms/step - loss: 0.4196 - accuracy: 0.8564 - val_loss: 0.6582 - val_accuracy: 0.7906
Epoch 7/50
1563/1563 [==============================] - 9s 6ms/step - loss: 0.3542 - accuracy: 0.8778 - val_loss: 0.6782 - val_accuracy

In [ ]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_24 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_24 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_25 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_25 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 16, 16, 32)       0   

In [ ]:
# Fit with data augmentation
batch_size = 32
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1,
                                                                 height_shift_range=0.1,
                                                                 horizontal_flip=True)

train_generator = data_generator.flow(x_train, y_train, batch_size)
steps_per_epoch = x_train.shape[0] // batch_size
r = model.fit(train_generator, validation_data=(x_test, y_test), steps_per_epoch=steps_per_epoch, epochs=50)

Epoch 1/50
1562/1562 [==============================] - 26s 16ms/step - loss: 1.4482 - accuracy: 0.4953 - val_loss: 1.0417 - val_accuracy: 0.6308
Epoch 2/50
1562/1562 [==============================] - 25s 16ms/step - loss: 1.0005 - accuracy: 0.6519 - val_loss: 0.9346 - val_accuracy: 0.6789
Epoch 3/50
1562/1562 [==============================] - 25s 16ms/step - loss: 0.8555 - accuracy: 0.7041 - val_loss: 0.7990 - val_accuracy: 0.7271
Epoch 4/50
1562/1562 [==============================] - 25s 16ms/step - loss: 0.7602 - accuracy: 0.7397 - val_loss: 0.7472 - val_accuracy: 0.7547
Epoch 5/50
1562/1562 [==============================] - 25s 16ms/step - loss: 0.6828 - accuracy: 0.7686 - val_loss: 0.7228 - val_accuracy: 0.7569
Epoch 6/50
1562/1562 [==============================] - 25s 16ms/step - loss: 0.6277 - accuracy: 0.7875 - val_loss: 0.6654 - val_accuracy: 0.7827
Epoch 7/50
1562/1562 [==============================] - 25s 16ms/step - loss: 0.5881 - accuracy: 0.8009 - val_loss: 0.6338 -

In [ ]:
plt.plot(r.history['loss'], label="loss")
plt.plot(r.history["val_loss"], label="val_loss")
plt.legend()

In [ ]:
plt.plot(r.history['accuracy'], label="accuracy")
plt.plot(r.history["val_accuracy"], label="val_acc")
plt.legend()

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(x_test).argmax(axis=1)
print(confusion_matrix(y_test, y_pred))

313/313 [==============================] - 1s 2ms/step
[[829   9  70  14  25   4  14   2  29   4]
 [  7 930   4   7   5   0  15   2  12  18]
 [ 28   2 781  28  79  18  53   6   5   0]
 [ 14   4  89 623  78  82  97   8   4   1]
 [  4   2  32  21 894   5  29  11   1   1]
 [ 13   2  68 105  66 677  48  15   5   1]
 [  5   0  29  22  14   6 920   1   2   1]
 [ 13   1  45  34  80  31  16 775   3   2]
 [ 31  12  17   3  12   2  12   2 900   9]
 [ 30  84  12   9   6   5  12   5  17 820]]


In [ ]:
from sklearn.metrics import confusion_matrix

# With Data Augmentation
y_pred = model.predict(x_test).argmax(axis=1)
print(confusion_matrix(y_test, y_pred))

313/313 [==============================] - 1s 3ms/step
[[886  11  17  10   5   3   6   8  27  27]
 [  3 966   0   0   0   0   2   0   5  24]
 [ 26   3 830  27  30  20  45   8   6   5]
 [ 10   4  26 698  28 119  66  25  11  13]
 [  3   2  17  22 872  12  42  27   2   1]
 [  3   2  10  65  22 839  18  30   3   8]
 [  3   3  16  24   3   5 938   3   1   4]
 [  1   1   6  14  12  20   3 935   2   6]
 [ 21  13   3   4   2   2   4   3 932  16]
 [  5  35   0   2   1   2   3   1  12 939]]
